In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Multilingual Sentiment-Aware AI Assistant**
This project implements a multilingual AI assistant designed to generate sentiment-aware responses using data from public social media posts (English and German). It offers two modes:

**Retrieval-Augmented Generation (RAG)** using sentence-transformer embeddings, FAISS vector search, and an open-source instruct model.

**Fine-tuned generative model (mT5-small)** trained directly on labeled data.

A Streamlit UI allows users to interact with both systems and compare their performance (A/B testing). The assistant is designed for use cases like customer feedback analysis, community moderation, and sentiment monitoring.

**praw stands for Python Reddit API Wrapper.**
It’s a Python library that makes it easy to access and scrape data from Reddit.


*Main reasons to install praw in your project:*

You want to collect Reddit posts or comments programmatically.

You can filter by subreddit, post type, upvotes, date, etc.

It lets you stream live data, or fetch historical posts.

It handles Reddit API authentication for you (OAuth2).

In [23]:
!pip install praw

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


**Here is the First step to download the comments from Reddit**


In [24]:
import praw
import json

reddit = praw.Reddit(
    client_id="5bX6wjvW6jVAPUs4NYTWZg",
    client_secret="kvcIOCVIU5N-Rv_kfR1Rs5zmGUwxgQ",
    user_agent="amg-chatbot-user-agentr"
)

def fetch_reddit_comments(subreddit, lang, limit=500):
    posts = []
    for comment in reddit.subreddit(subreddit).comments(limit=limit):
        text = comment.body.strip()
        if len(text) > 20:
            posts.append({
                "text": text,
                "language": lang,
                "source": f"reddit:{subreddit}",
                "sentiment": None
            })
    return posts

# Example: English + German
en_posts = fetch_reddit_comments("technology", "en", 500)
de_posts = fetch_reddit_comments("de", "de", 500)

with open("reddit_praw_posts.jsonl", "w", encoding="utf-8") as f:
    for post in en_posts + de_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print(f"Saved {len(en_posts) + len(de_posts)} Reddit posts!")


Saved 956 Reddit posts!


Then Open the Reddit file and then do the sentiment analysis based on the input
1. Load your Reddit posts
2. Auto-label sentiment
3. Save labeled data


In [25]:
from transformers import pipeline
import json

# Load your Reddit posts
def load_posts(file_path):
    posts = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            posts.append(json.loads(line.strip()))
    return posts

# Auto-label sentiment
def label_sentiment(posts, limit=2000):
    sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    labeled = []
    for post in posts[:limit]:
        result = sentiment_analyzer(post["text"][:512])[0]  # Truncate long texts
        stars = int(result["label"].split()[0])  # e.g. "4 stars" -> 4
        if stars >= 4:
            sent = "positive"
        elif stars == 3:
            sent = "neutral"
        else:
            sent = "negative"
        post["sentiment"] = sent
        labeled.append(post)
    return labeled

# Save labeled data
def save_labeled(posts, out_file):
    with open(out_file, "w", encoding="utf-8") as f:
        for post in posts:
            f.write(json.dumps(post, ensure_ascii=False) + "\n")
    print(f"✅ Saved {len(posts)} labeled posts to {out_file}")

# Example
reddit_posts = load_posts("reddit_praw_posts.jsonl")
labeled_posts = label_sentiment(reddit_posts, limit=2000)
save_labeled(labeled_posts, "reddit_labeled_posts.jsonl")


Device set to use cuda:0


✅ Saved 956 labeled posts to reddit_labeled_posts.jsonl


# **Now Loading the Youtube Comments**


In [26]:
import json

def load_yt_comments(file_path):
    with open(file_path, encoding="utf-8") as f:
        data = json.load(f)

    posts = []
    for c in data.get("comments", []):
        text = c.get("text", "").strip()
        if len(text) > 15:  # skip super short stuff
            posts.append({
                "text": text,
                "language": "en",  # adjust if you know comments are German
                "source": "youtube",
                "sentiment": None
            })
    return posts

yt_posts = load_yt_comments("New AMG GT review： Better than a 911？! [2vWh2kaeZkI].info.json")
print(f"✅ Loaded {len(yt_posts)} YouTube comments")



✅ Loaded 2761 YouTube comments


# Combining the reddit posts and the youtube posts together into one file for the sentiment analysis

In [27]:
import json

# Load Reddit posts
with open("reddit_praw_posts.jsonl", encoding="utf-8") as f:
    reddit_posts = [json.loads(line.strip()) for line in f]

# Now combine
combined_posts = reddit_posts + yt_posts
print(f"🧮 Combined total posts: {len(combined_posts)}")

# Save to file
with open("all_social_posts.jsonl", "w", encoding="utf-8") as f:
    for post in combined_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print("✅ Saved combined dataset as all_social_posts.jsonl")


🧮 Combined total posts: 3717
✅ Saved combined dataset as all_social_posts.jsonl


In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import json

# Load model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

def get_sentiment(text):
    encoded = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    output = model(**encoded)
    scores = softmax(output.logits.detach().numpy()[0])
    labels = ["negative", "neutral", "positive"]
    return labels[scores.argmax()]


# Load combined data
with open("all_social_posts.jsonl", encoding="utf-8") as f:
    posts = [json.loads(line.strip()) for line in f]

# Label ~2,000 posts
labeled_posts = []
for post in posts[:2000]:
    text = post.get("text", "")
    sentiment = get_sentiment(text)
    post["sentiment"] = sentiment
    labeled_posts.append(post)

# Save
with open("labeled_social_posts.jsonl", "w", encoding="utf-8") as f:
    for post in labeled_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print(f"✅ Labeled and saved {len(labeled_posts)} posts")


✅ Labeled and saved 2000 posts


**sentence-transformers**
A library that provides pre-trained models to generate dense vector embeddings for text.

These embeddings capture the semantic meaning of text (so “Hello” and “Hi” get similar vectors).

Multilingual model (paraphrase-multilingual-MiniLM-L12-v2) supports both English and German, matching your project’s requirements.

👉 You use it to:

Convert each post/comment into a numerical vector.

Store those vectors in a database (e.g. FAISS).

Later, retrieve similar posts by comparing vectors to a user’s query vector.

**faiss-cpu**
FAISS = Facebook AI Similarity Search

It lets you efficiently search through large collections of vectors to find the nearest neighbors (e.g. top-k relevant posts).

You use it here to:

Store all your post embeddings.

Quickly retrieve posts semantically similar to a query.

📌 **Why this model?**
**'paraphrase-multilingual-MiniLM-L12-v2':**

Designed for generating sentence embeddings.

Multilingual: works well for both English and German (perfect for your dataset).

Compact (MiniLM = small + fast), good for prototyping.



In [29]:
# Install sentence-transformers if not already done
!pip install -U sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import json

# Load model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


**Below code does the following :**


1. Loading your social media dataset
2. Filtering / cleaning posts
3. Embedding posts into dense vector form
4. Building a FAISS index for fast semantic search
5. Saving the index + texts for later querying


In [30]:
import json
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load posts
with open("all_social_posts.jsonl", encoding="utf-8") as f:
    posts = [json.loads(line.strip()) for line in f]

# Extract + clean
texts = [post.get("text", "") for post in posts]
clean_texts = [t.strip() for t in texts if len(t.strip()) > 30]
print(f"✅ Filtered down to {len(clean_texts)} clean posts.")

# Embed
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(clean_texts, batch_size=64, show_progress_bar=True).astype("float32")

# Build index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# Save
faiss.write_index(index, "rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "wb") as f:
    pickle.dump(clean_texts, f)

print("✅ Clean FAISS index and texts saved.")


✅ Filtered down to 3194 clean posts.


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

✅ Clean FAISS index and texts saved.


**Next task is :**


1. load a pre-trained generative LLM (FLAN-T5 Large in this case) that will:Take the retrieved snippets + query as input.Generate a natural language answer.
2. Your FAISS index: stores embeddings of your posts.Your list of clean texts: for mapping FAISS index positions back to actual post text.
3. Embeds the query (using your embedding model — note: you passed model which is actually your sentence-transformer in your case).
4. Searches FAISS index for k most similar embeddings.Returns the corresponding text snippets.
5. Creates a prompt that includes retrieved snippets + query.
6. Tokenizes and passes to your generator (FLAN-T5).
7. Generates a response capped at max_tokens tokens.
8. Returns the decoded response the retrieved refs.



In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load generator
gen_model_name = "google/flan-t5-large"  # You can change to flan-t5-large or another
tokenizer_gen = AutoTokenizer.from_pretrained(gen_model_name)
model_gen = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

# Load index + texts
index = faiss.read_index("rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "rb") as f:
    clean_texts = pickle.load(f)

# Function: retrieve
def retrieve_top_k(query, embed_model, index, texts, k=5):
    query_vec = embed_model.encode([query]).astype("float32")
    D, I = index.search(query_vec, k)
    return [texts[i] for i in I[0]]

# Function: generate
def generate_answer(query, retrieved, tokenizer, model, max_tokens=128):
    context = "\n".join(retrieved)
    prompt = (
        f"Here are some customer comments:\n"
        f"{context}\n"
        f"Based on these, provide a clear, helpful, natural-language answer to the question:\n"
        f"{query}"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    output = model.generate(**inputs, max_length=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True), retrieved

# Example usage
query = "What do customers think about the AMG GT XX?"
retrieved = retrieve_top_k(query, model, index, clean_texts, k=5)
answer, refs = generate_answer(query, retrieved, tokenizer_gen, model_gen)

print(f"🔹 Answer: {answer}\n")
print("🔹 Retrieved snippets:")
for r in refs:
    print("-", r[:300])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔹 Answer: It's an SL with AMG branding and nothing more.

🔹 Retrieved snippets:
- Imagine thinking this is prettier than the original 2016 AMG GT.
- That new AMG GT looks amazing! 😎👍🏾. I’m looking forward to seeing it in a drag race. 😎
- I can alleviate a lot of the confusion in the comments section. This is not a true AMG GT. It’s an SL with AMG branding and nothing more. These car reviewers have to maintain good relationships with the manufacturers so you’ll hear them skim over that like Matt did here.

Real AMG GT: SLS based, RWD
- Thanks for the review, encouraged me to make the decision to shortlist the new AMG GT63. I'm gonna take it for a test drive next time I go to sleep and have a dream.
- nobody really buys an amg gt for daily driving , with that said i dont think this is an improvement on the last gen , it doesnt look necessarily better , its not more powerfull , its 400kg heavier and the interior isnt that much better either . I think the older generation car will apreciat


**Now,It generates a large set of diverse natural-language queries**

👉 The code builds:

**
Extra queries for comparisons between models (e.g. “Which is better: AMG GT or BMW M3?”).

➡ This ensures wide coverage of topics customers might ask about.

*2️.It retrieves relevant posts using semantic search (FAISS)*
👉 For each query:

The code encodes the query into an embedding (using paraphrase-multilingual-MiniLM-L12-v2).

It searches your FAISS index to find the top 5 most relevant social posts.

These retrieved posts form the context for generating an answer.

*3️. It generates an answer grounded in retrieved content (RAG)*
👉 The system:

Builds a prompt combining the query + retrieved context.

Feeds this prompt to FLAN-T5-base to generate a natural-language answer.

The generator is guided to answer based on the retrieved posts (this is the core of RAG).

*4️. It saves query-answer pairs as training data*
👉 Each query + generated answer is saved as:

json
Copy
Edit
{"input": "query", "target": "generated answer"}
➡ The file (qa_finetune_data_instructlab2.jsonl) can be used to fine-tune a model (e.g. mT5) or train downstream tasks.

*5️.It robustly handles errors and large-scale generation*
👉 Uses tqdm to track progress.
👉 Catches exceptions so errors in a single query don’t stop the process.
👉 Designed to generate thousands of Q+A pairs efficiently.

In [32]:
import json
import pickle
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- Load models ---
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
gen_model_name = "google/flan-t5-base"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

# --- Load FAISS + texts ---
index = faiss.read_index("rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "rb") as f:
    texts = pickle.load(f)

# --- Topics + aspects ---
topics = [
    # AMG models
    "AMG GT XX", "AMG GT", "AMG GT Black Series", "AMG C63", "AMG E63", "AMG A45",
    "AMG GLC 63", "AMG GLE 63", "AMG S63", "AMG CLA 45", "AMG SL63",
    # BMW
    "BMW M3", "BMW M5", "BMW X5M", "BMW i4", "BMW iX", "BMW X3", "BMW M2",
    # Audi
    "Audi RS6", "Audi RS7", "Audi S3", "Audi Q8", "Audi e-tron", "Audi RS Q8",
    # Porsche
    "Porsche 911", "Porsche Taycan", "Porsche Macan", "Porsche Cayenne", "Porsche Panamera"
]

aspects = [
    "What do customers think about", "What do people dislike about",
    "What do customers like about", "Is it worth the money?",
    "What do people say about the design of", "What are common complaints about",
    "What positive feedback is there for", "What negative feedback is there for",
    "How do people feel about the price of", "How reliable is",
    "How is the comfort of", "What about the safety features of",
    "Is the tech in", "What about fuel efficiency of",
    "Is it fun to drive", "How does the interior of",
    "What about the build quality of", "How does the handling feel for",
    "How practical is", "What about resale value of"
]

# --- Build queries ---
queries = [f"{aspect} {topic}?" for topic in topics for aspect in aspects]

# Add comparison queries
compare_templates = [
    "How does {} compare to {}?",
    "Which is better: {} or {}?",
    "Why choose {} over {}?"
]

for i, a in enumerate(topics):
    for b in topics[i+1:]:
        for tmpl in compare_templates:
            queries.append(tmpl.format(a, b))

print(f"✅ Total queries to generate: {len(queries)}")

# --- Generate Q+A pairs ---
output_file = "qa_finetune_data_instructlab2.jsonl"
with open(output_file, "w", encoding="utf-8") as f_out:
    for query in tqdm(queries, desc="Generating Q+A pairs"):
        try:
            query_vec = embed_model.encode([query]).astype("float32")
            D, I = index.search(query_vec, k=5)
            retrieved = [texts[i] for i in I[0]]
            context = " ".join(retrieved)

            prompt = f"Question: {query}\nContext: {context}\nAnswer:"
            inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            outputs = gen_model.generate(**inputs, max_length=100)
            answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

            record = {"input": query, "target": answer}
            f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"⚠️ Skipped query due to error: {query}\nError: {e}")

print(f"✅ Saved: {output_file}")


✅ Total queries to generate: 1798


Generating Q+A pairs:   0%|          | 0/1798 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Q+A pairs:   0%|          | 1/1798 [00:00<27:46,  1.08it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Q+A pairs: 100%|█████████▉| 1797/1798 [29:24<00:01,  1.10s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Q+A pairs: 100%|██████████| 1798/1798 [29:25<00:00,  1.02it/s]

✅ Saved: qa_finetune_data_instructlab2.jsonl


In [33]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


1️⃣ It loads and prepares the fine-tuning dataset
👉 The code reads your generated Q+A pairs from qa_finetune_data_instructlab2.jsonl and converts them into a Hugging Face Dataset.
👉 Each pair includes an "input" (query) and a "target" (answer).

2️⃣ It tokenizes the data for the model
👉 The preprocess function:

Tokenizes queries (inputs) and answers (labels).

Pads + truncates to fixed lengths (512 for input, 100 for target).

Prepares the data for seq2seq training (input → target mapping).

3️⃣ It configures and runs fine-tuning of FLAN-T5-base
👉 Sets up Seq2SeqTrainer with:

Small batch size (1)

3 training epochs

Save checkpoints + logging
👉 Starts training FLAN-T5-base on your Q+A dataset so the model learns to generate answers without retrieval.

4️⃣ It saves model checkpoints during training
👉 The fine-tuned model will be stored in ./flan_t5_finetuned_output1 for later use (e.g. evaluation or deployment).



In [1]:
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.metrics import f1_score
import time

# 1️⃣ Load Q+A data
data = []
with open("qa_finetune_data_instructlab2.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
print(f"Loaded {len(data)} Q+A pairs.")

dataset = Dataset.from_list(data)

# 2️⃣ Load model + tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 3️⃣ Preprocess
def preprocess(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["target"], padding="max_length", truncation=True, max_length=100)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

# 4️⃣ Training config
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_finetuned_output1",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    report_to=[],
    remove_unused_columns=False
)

# 5️⃣ Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# 6️⃣ Train
trainer.train()






2025-07-07 06:20:42.014392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751869242.186005     353 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751869242.234841     353 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loaded 1798 Q+A pairs.


Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

/tmp/ipykernel_353/3894131832.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,13.221200
200,1.478400
300,0.437600
400,0.505400
500,0.369000
600,0.311800
700,0.326400
800,0.364700
900,0.217000
1000,0.270200


TrainOutput(global_step=5394, training_loss=0.4734006746636526, metrics={'train_runtime': 899.5486, 'train_samples_per_second': 5.996, 'train_steps_per_second': 5.996, 'total_flos': 3693581159104512.0, 'train_loss': 0.4734006746636526, 'epoch': 3.0})

In [ ]:
!zip -r flan_t5_finetuned_output1.zip flan_t5_finetuned_output1

1️⃣ Loads your fine-tuned model + dataset
👉 The code loads your best fine-tuned FLAN-T5 checkpoint and the Q+A dataset used for evaluation.

2️⃣ Prepares data and runs predictions
👉 It tokenizes the data, then uses Seq2SeqTrainer to generate model predictions (no further training).
👉 It measures latency — how long each query takes on average to process.

3️⃣ Decodes predictions + evaluates F1 score
👉 Decodes the model’s answers to text.
👉 Compares generated answers to references at a basic word-level, computing a micro F1 score for indicative accuracy.

4️⃣ Reports efficiency + transparency metrics
👉 Calculates total tokens generated + runtime cost estimate per 1k tokens.
👉 Notes transparency: since this is a fine-tuned model (no retrieval), no source references are provided.
👉 Highlights maintainability: no DB updates needed; re-training required for updates.

In [ ]:
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import time
from sklearn.metrics import f1_score

# 1️⃣ Load your fine-tuned model + tokenizer
model_dir = "./flan_t5_finetuned_output1/checkpoint-5394"  # replace xxxx with your best checkpoint folder
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# 2️⃣ Load your data
data = []
with open("qa_finetune_data_instructlab2.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
print(f"✅ Loaded {len(data)} Q+A pairs.")

dataset = Dataset.from_list(data)

# 3️⃣ Preprocess
def preprocess(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["target"], padding="max_length", truncation=True, max_length=100)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

# 4️⃣ Prepare trainer (no training — just for predict)
training_args = Seq2SeqTrainingArguments(
    output_dir="./tmp_eval_output",
    per_device_eval_batch_size=4,
    report_to=[],
    remove_unused_columns=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)

# 5️⃣ Run prediction + measure latency
sample_eval = dataset.select(range(min(150, len(dataset))))
sample_tokenized = tokenized_dataset.select(range(min(300, len(dataset))))

start_time = time.time()
predictions = trainer.predict(sample_tokenized)
latency = (time.time() - start_time) / len(sample_eval)

# 6️⃣ Decode outputs
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_refs = [ex["target"] for ex in sample_eval]

# Show a few examples
for i in range(3):
    print(f"🔹 Q: {sample_eval[i]['input']}")
    print(f"🔹 Ref: {decoded_refs[i]}")
    print(f"🔹 Pred: {decoded_preds[i]}\n")

# 7️⃣ F1-score (basic micro word-level — for indicative purposes)
ref_words = [ref.split() for ref in decoded_refs]
pred_words = [pred.split() for pred in decoded_preds]

ref_flat = sum(ref_words, [])
pred_flat = sum(pred_words, [])

if len(pred_flat) < len(ref_flat):
    pred_flat += ["<pad>"] * (len(ref_flat) - len(pred_flat))

f1 = f1_score(ref_flat, pred_flat, average='micro', zero_division=1)
print(f"✅ F1-score (micro word-level): {f1:.4f}")

# 8️⃣ Runtime cost
tokens_generated = sum(len(tokenizer.encode(pred)) for pred in decoded_preds)
print(f"✅ ~{tokens_generated} tokens generated")
print(f"✅ Runtime cost estimate: {tokens_generated / 1000:.2f} per 1k tokens")

# 9️⃣ Latency
print(f"✅ Avg latency per query: {latency:.4f} sec")

# 10️⃣ Transparency + maintainability
print("ℹ️ Transparency N/A — this is a fine-tuned model, no sources returned")
print("ℹ️ Maintainability: re-train required for major updates; no DB maintenance needed.")


✅ Loaded 1798 Q+A pairs.


Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

/tmp/ipykernel_353/1337442726.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


**This pipeline collects and structures ~3,000–5,000 multilingual social posts, builds a FAISS-based RAG system using sentence-transformer embeddings, and fine-tunes a FLAN-T5 model on synthetic Q+A pairs. It supports A/B testing between retrieval-augmented and direct generation approaches via a simple Streamlit UI. Evaluation measures accuracy (F1), latency, runtime cost, transparency (for RAG), and maintainability. The setup enables flexible, multilingual customer feedback analysis with both updatable and static model options.*# ***